In [13]:
import boto3
import datetime
import pytz

In [14]:
s3 = boto3.resource('s3')
s3client = boto3.client('s3')

In [15]:
target_bucket = 'aii-archive'
#target_bucket = 'wcc-tacc-test'


In [24]:
target_max_date = datetime.datetime(2020,9,30,0,0,0,0, pytz.UTC)
target_min_size = 1024 * 1024 # 1 MB
old_file_set = []

In [21]:
archive_tag = 'deep_archive'

In [17]:
def set_archive_tag(bucket, key):
    response = s3.meta.client.put_object_tagging(
        Bucket=bucket,
        Key=key,
        Tagging={
            'TagSet': [
                {
                    'Key': 'storage_level',
                    'Value': archive_tag,
                    },
                ]
            },
        )

In [18]:
count = 0
limit = 100000000000

small_count = 0
large_count = 0
large_old_count = 0


paginator = s3client.get_paginator('list_objects')
pageresponse = paginator.paginate(Bucket=target_bucket)
#out_f.write('+++++\n')
#out_f.write(str(bucket) + '\n')
#out_f.write('+++++\n')

# PageResponse Holds 1000 objects at a time and
# will continue to repeat in chunks of 1000.
for pageobject in pageresponse:
    #out_f.write('----\n')
    #out_f.write('----\n')
    if 'Contents' in pageobject:
        for item in pageobject["Contents"]:
            # Skip small files
            if item['Size'] < target_min_size:
                small_count += 1
                continue
            large_count += 1
            if item['LastModified'] <= target_max_date:
                #print("found")
                #print(item)
                set_archive_tag(target_bucket, item['Key'])
                large_old_count +=1 
            else: 
                continue    
            count += 1
            if count > limit:
                break
        if count > limit:
            break
    if count > limit:
        break

In [19]:
print(small_count, large_count, large_old_count)

179409 14834 14834


In [25]:
count = 0
limit = 1000000000
tagged_count = 0
not_tagged_count = 0
total_count = 0
tagged_list = []

paginator = s3client.get_paginator('list_objects')
pageresponse = paginator.paginate(Bucket=target_bucket)
#out_f.write('+++++\n')
#out_f.write(str(bucket) + '\n')
#out_f.write('+++++\n')

# PageResponse Holds 1000 objects at a time and
# will continue to repeat in chunks of 1000.
tagged_size = 0
for pageobject in pageresponse:
    #out_f.write('----\n')
    #out_f.write('----\n')
    if 'Contents' in pageobject:
        for item in pageobject["Contents"]:
            not_tagged_count += 1
            total_count += 1
            # Skip small files
            if item['Size'] < target_min_size:
                continue

            if item['LastModified'] <= target_max_date:
                tags = s3.meta.client.get_object_tagging(
                    Bucket=target_bucket,
                    Key=item['Key']
                    )
                #print("found")
                if 'TagSet' in tags:
                    if tags['TagSet']:
                        if tags['TagSet'][0]['Value'] != archive_tag:
                            print("Missed File")
                        else:
                            tagged_size += item['Size']
                            tagged_count += 1
                        tagged_list.append(
                            tuple([
                                target_bucket,
                                item['Key'],
                                item['Size'],
                                item['StorageClass'],
                                item['LastModified']]
                            ))
                            
                not_tagged_count -= 1
                #print(item)
                #print(tags)
      


            

                

            count += 1
            if count > limit:
                break
        if count > limit:
            break
    if count > limit:
        break

In [26]:
tagged_list[:10]



[('aii-archive',
  'SpectrumSight/Arlington/ARL-ALL_ss_0403-04192020.xlsx',
  45919269,
  'STANDARD',
  datetime.datetime(2020, 6, 11, 19, 25, 33, tzinfo=tzutc())),
 ('aii-archive',
  'SpectrumSight/Arlington/ARL-ALL_ss_04202020.csv',
  13329415,
  'STANDARD',
  datetime.datetime(2020, 6, 11, 19, 25, 33, tzinfo=tzutc())),
 ('aii-archive',
  'SpectrumSight/Arlington/ARL-ALL_ss_04202020.xlsx',
  10920959,
  'STANDARD',
  datetime.datetime(2020, 6, 11, 19, 25, 33, tzinfo=tzutc())),
 ('aii-archive',
  'SpectrumSight/Arlington/ARL-ALL_ss_04212020.csv',
  27093229,
  'STANDARD',
  datetime.datetime(2020, 6, 11, 19, 25, 33, tzinfo=tzutc())),
 ('aii-archive',
  'SpectrumSight/Arlington/ARL-ALL_ss_04212020.xlsx',
  22179823,
  'STANDARD',
  datetime.datetime(2020, 6, 11, 19, 25, 33, tzinfo=tzutc())),
 ('aii-archive',
  'SpectrumSight/Arlington/ARL-ALL_ss_04222020.csv',
  46948958,
  'STANDARD',
  datetime.datetime(2020, 6, 11, 19, 25, 33, tzinfo=tzutc())),
 ('aii-archive',
  'SpectrumSight/Arli

In [27]:
len(tagged_list)

14834

In [28]:
tags['TagSet']

[{'Key': 'storage_level', 'Value': 'deep_archive'}]

In [29]:
tags['TagSet'][0]['Value']
                      

'deep_archive'

In [30]:
tagged_size

816613856594